In [1]:
! pip install nltk

In [2]:
import numpy as np
import pandas as pd
import pickle

import random
import nltk

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [3]:
text_df=pd.read_csv("C:/Users/HP/Downloads/fake_or_real_news.csv")

In [4]:
text_df

,id,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [5]:
text = list(text_df.text.values)
joined_txt=" ".join(text) #new string of only text
print(len(joined_txt))

29826765


In [23]:
partial_txt = joined_txt[:100000]

In [24]:
tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+") #only words
tokens=tokenizer.tokenize(partial_txt.lower()) # only lower case words are tokenized


In [25]:
tokens

['daniel',
 'greenfield',
 'a',
 'shillman',
 'journalism',
 'fellow',
 'at',
 'the',
 'freedom',
 'center',
 'is',
 'a',
 'new',
 'york',
 'writer',
 'focusing',
 'on',
 'radical',
 'islam',
 'in',
 'the',
 'final',
 'stretch',
 'of',
 'the',
 'election',
 'hillary',
 'rodham',
 'clinton',
 'has',
 'gone',
 'to',
 'war',
 'with',
 'the',
 'fbi',
 'the',
 'word',
 'unprecedented',
 'has',
 'been',
 'thrown',
 'around',
 'so',
 'often',
 'this',
 'election',
 'that',
 'it',
 'ought',
 'to',
 'be',
 'retired',
 'but',
 'it',
 's',
 'still',
 'unprecedented',
 'for',
 'the',
 'nominee',
 'of',
 'a',
 'major',
 'political',
 'party',
 'to',
 'go',
 'war',
 'with',
 'the',
 'fbi',
 'but',
 'that',
 's',
 'exactly',
 'what',
 'hillary',
 'and',
 'her',
 'people',
 'have',
 'done',
 'coma',
 'patients',
 'just',
 'waking',
 'up',
 'now',
 'and',
 'watching',
 'an',
 'hour',
 'of',
 'cnn',
 'from',
 'their',
 'hospital',
 'beds',
 'would',
 'assume',
 'that',
 'fbi',
 'director',
 'james',
 'c

In [26]:
unique_tokens=np.unique(tokens)
unique_token_index = {token : idx for idx, token in enumerate(unique_tokens)}

In [27]:
unique_token_index

{'0': 0,
 '000': 1,
 '1': 2,
 '10': 3,
 '100': 4,
 '106': 5,
 '10th': 6,
 '11': 7,
 '12': 8,
 '12pm': 9,
 '13': 10,
 '14': 11,
 '15': 12,
 '16': 13,
 '160': 14,
 '17': 15,
 '187': 16,
 '19': 17,
 '1939': 18,
 '1960': 19,
 '197': 20,
 '1971': 21,
 '1972': 22,
 '1983': 23,
 '1984': 24,
 '1985': 25,
 '1990s': 26,
 '1992': 27,
 '1994': 28,
 '1996': 29,
 '1998': 30,
 '1999': 31,
 '2': 32,
 '20': 33,
 '200': 34,
 '2000': 35,
 '20001': 36,
 '2001': 37,
 '2002': 38,
 '2003': 39,
 '2004': 40,
 '2006': 41,
 '2007': 42,
 '2008': 43,
 '2009': 44,
 '2010': 45,
 '2011': 46,
 '2012': 47,
 '2013': 48,
 '2014': 49,
 '2015': 50,
 '2016': 51,
 '2020': 52,
 '21': 53,
 '2117': 54,
 '21st': 55,
 '21wire': 56,
 '22': 57,
 '227': 58,
 '23': 59,
 '24': 60,
 '241': 61,
 '27': 62,
 '2nd': 63,
 '3': 64,
 '30': 65,
 '300': 66,
 '31': 67,
 '33': 68,
 '3pm': 69,
 '4': 70,
 '40': 71,
 '400': 72,
 '42': 73,
 '49': 74,
 '5': 75,
 '50': 76,
 '500': 77,
 '51': 78,
 '52': 79,
 '57': 80,
 '5pm': 81,
 '6': 82,
 '60': 83,
 '

In [29]:
n_words = 10
input_words = []
next_word = []

for i in range(len(tokens)- n_words):
  input_words.append(tokens[i:i+n_words])
  next_word.append(tokens[i+n_words]) 

In [30]:
X=np.zeros((len(input_words),n_words,len(unique_tokens)), dtype=bool)
y=np.zeros((len(input_words),len(unique_tokens)), dtype=bool)

In [31]:
for i,words in enumerate(input_words):
  for j,word in enumerate(words):
    X[i,j, unique_token_index[word]] = 1
  y[i ,unique_token_index[next_word[i]]] = 1

In [32]:
model=Sequential([
  LSTM(128 , input_shape=(n_words,len(unique_tokens)), return_sequences=True),
  LSTM(128),
  Dense(len(unique_tokens)),
  Activation('softmax')
])

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [33]:
model.compile(optimizer=RMSprop(learning_rate=0.01),loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X,y, batch_size=128, epochs=10, shuffle=True)

Epoch 1/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 15s 95ms/step - accuracy: 0.0511 - loss: 7.1194
Epoch 2/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 13s 100ms/step - accuracy: 0.0586 - loss: 6.6571
Epoch 3/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 14s 106ms/step - accuracy: 0.0658 - loss: 6.5482
Epoch 4/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 15s 113ms/step - accuracy: 0.0739 - loss: 6.3646
Epoch 5/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 15s 114ms/step - accuracy: 0.0812 - loss: 6.1710
Epoch 6/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 16s 115ms/step - accuracy: 0.0985 - loss: 5.9322
Epoch 7/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 16s 119ms/step - accuracy: 0.1166 - loss: 5.6324
Epoch 8/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 16s 121ms/step - accuracy: 0.1311 - loss: 5.4056
Epoch 9/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 17s 125ms/step - accuracy: 0.1580 - loss: 5.1322
Epoch 10/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 17s 123ms/step - accuracy: 0.1840 - loss: 4.8489


In [17]:
model.save('Next_word.keras')

In [34]:
def pred_next(input_text,n_best):
  input_txt=input_text.lower()
  X= np.zeros((1,n_words,len(unique_tokens)))
  for j,word in enumerate(input_txt.split()):
    X[0,j, unique_token_index[word]] = 1

    prediction=model.predict(X)[0]
    possible = np.argpartition(prediction,-n_best)[-n_best:]
    return ([unique_tokens[idx] for idx in possible])

In [35]:
text_2 = "He told her not to visit there but she didn't"
hhh = pred_next(text_2 , 10)
print(hhh)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
['care', 'on', 'from', 'republicans', 'of', 'when', 'asked', 'i', 'with', 'years']
